In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook

# Load data from the first Excel workbook
wb_location1 = load_workbook("StaffPerformanceOverviewCR06-23-23 test copy.xlsx")
ws_location1 = wb_location1.active

# Read the data from the worksheet into a Pandas DataFrame
data_range_location1 = ws_location1['A3:D11']  # Assuming data for location 1 is in column A starting from row 2
data_location1 = [[cell.value for cell in row] for row in data_range_location1]
df_location1 = pd.DataFrame(data_location1, columns=['EmployeeID', 'Data1', 'Data2', 'Data3'])

# Load data from the second Excel workbook
wb_location2 = load_workbook("StaffPerformanceOverviewNH06-23-23 test copy.xlsx")
ws_location2 = wb_location2.active

# Read the data from the worksheet into a Pandas DataFrame
data_range_location2 = ws_location2['A3:D9']  # Assuming data for location 2 is in column A starting from row 2
data_location2 = [[cell.value for cell in row] for row in data_range_location2]
df_location2 = pd.DataFrame(data_location2, columns=['EmployeeID', 'Data1', 'Data2', 'Data3'])

# Consolidate data based on EmployeeID and sum the values for each location
consolidated_data_location1 = df_location1.groupby('EmployeeID', as_index=False).sum()
consolidated_data_location2 = df_location2.groupby('EmployeeID', as_index=False).sum()

# Merge the consolidated data for employees working at both locations (inner merge)
consolidated_data_both_locations = pd.merge(consolidated_data_location1, consolidated_data_location2, on='EmployeeID', suffixes=('_loc1', '_loc2'), how='inner')
consolidated_data_both_locations['Data1'] = consolidated_data_both_locations['Data1_loc1'] + consolidated_data_both_locations['Data1_loc2']
consolidated_data_both_locations['Data2'] = consolidated_data_both_locations['Data2_loc1'] + consolidated_data_both_locations['Data2_loc2']
consolidated_data_both_locations['Data3'] = consolidated_data_both_locations['Data3_loc1'] + consolidated_data_both_locations['Data3_loc2']

# Remove the unnecessary columns from the consolidated DataFrame
consolidated_data_both_locations.drop(columns=['Data1_loc1', 'Data1_loc2', 'Data2_loc1', 'Data2_loc2', 'Data3_loc1', 'Data3_loc2'], inplace=True)

# Convert the consolidated data to a list
consolidated_data_list = consolidated_data_both_locations.values.tolist()

# Create a new worksheet in the first workbook to place the consolidated data
new_ws = wb_location1.create_sheet(title='Consolidated Data')

# Determine the starting row for placing the consolidated data
start_row = 2  # Assuming you want to start placing the consolidated data in row 2

# Put the consolidated data into the new worksheet
for row_index, row_data in enumerate(consolidated_data_list):
    for col_index, cell_value in enumerate(row_data):
        cell = new_ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the first updated workbook with the new worksheet
wb_location1.save("mergeddata.xlsx")
